In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchinfo import summary 
import numpy as np
import matplotlib.pyplot as plt
import pprint

from CharToIndex import CharToIndex
from MyDatasets import BaseDataset as MyDataset
from MyCustomLayer import CharToVectorLayer 


import time
import math


In [2]:
chars_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/main/all_chars.npy"
tokens = CharToIndex(chars_file_path)
data_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/main/origin.npy"
data = np.load(data_file_path,allow_pickle=True)

EMBEDDING_DIM = 10
HIDDEN_SIZE = 300
BATCH_SIZE = 1
VOCAB_SIZE = len(tokens)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = MyDataset(data,chars_file_path,device=device)

def split_dataset(dataest,train_valid_ratio=None):
    assert type(train_valid_ratio)==tuple
    assert type(train_valid_ratio[0])==int and type(train_valid_ratio[1])==int 
    assert train_valid_ratio[0]+train_valid_ratio[1]==10 

    whole_length = len(dataset)
    indices_sets = []
    pre_point = 0
    split_num = 10
    sets_length = int(whole_length/split_num)
    valid_ratio = train_valid_ratio[1]

    for i in range(split_num):
        indices_sets.append(list(range(pre_point,pre_point+sets_length)))
        pre_point+=sets_length

    import random
    import itertools
    random_sets = random.sample(indices_sets,split_num) 
    valid_indices = list(itertools.chain.from_iterable(random_sets[:valid_ratio]))
    train_indices = list(itertools.chain.from_iterable(random_sets[valid_ratio:]))

    from torch.utils.data import Subset
    valid_dataset = Subset(dataset,valid_indices)
    train_dataset = Subset(dataset,train_indices)
    
    return train_dataset,valid_dataset


train_dataset, valid_dataset = split_dataset(dataset,train_valid_ratio=(9,1)) #データセットを訓練、検証用に分割
train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True) #訓練データのみシャッフル
valid_dataloader=DataLoader(valid_dataset,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)

In [60]:
class CharToVectorLayer3(nn.Module):
    def __init__(self):
        super().__init__()
        self.vec_of_char = torch.load("/net/nfs2/export/home/ohno/CR_pytorch/data/main/char2vec")
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self,x):
        #(1,5,10)
        output = []
        for timestep_item in x:
            timesteps_list = []
            for feature_item in timestep_item:
                features_list = []
                for f in feature_item:
                    features_list.append(self.vec_of_char[f.item()])
                timesteps_list.append(torch.stack(features_list,0))
            output.append(torch.stack(timesteps_list,0))
        
        return torch.stack(output,0).to(self.device)

In [61]:
layer = CharToVectorLayer3()
for x,y in valid_dataloader:
    output = layer(x)
    break

torch.Size([1, 5, 10, 5])


In [62]:
batch_size=64
timesteps_size=5
features_size=10
vector_size = 5
m = nn.Linear(vector_size, 1)
input = torch.randn(batch_size,timesteps_size,features_size,vector_size)
print(f'input size:{input.size()}')
output = m(input)
print(output.size())
print(torch.squeeze(output).size())

input size:torch.Size([64, 5, 10, 5])
torch.Size([64, 5, 10, 1])
torch.Size([64, 5, 10])
